# DCF
#### Alpha Vantage

In [1]:
import requests
import json


AV_API_KEY = "WGD32KDX07IA26F3"
TICKER = "UBER"

In [2]:
# income_statements_resp = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={TICKER}&apikey={AV_API_KEY}").json()
# balance_sheets_resp = requests.get(f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={TICKER}&apikey={AV_API_KEY}").json()

In [3]:
# income_statements_resp["annualReports"][0]

{'fiscalDateEnding': '2023-12-31',
 'reportedCurrency': 'USD',
 'grossProfit': '7236000000',
 'totalRevenue': '37281000000',
 'costOfRevenue': '30045000000',
 'costofGoodsAndServicesSold': '22457000000',
 'operatingIncome': '1110000000',
 'sellingGeneralAndAdministrative': '7038000000',
 'researchAndDevelopment': '3164000000',
 'operatingExpenses': '10844000000',
 'investmentIncomeNet': '484000000',
 'netInterestIncome': '-633000000',
 'interestIncome': '484000000',
 'interestExpense': '633000000',
 'nonInterestIncome': '-182000000',
 'otherNonOperatingIncome': '-198000000',
 'depreciation': '355000000',
 'depreciationAndAmortization': '550000000',
 'incomeBeforeTax': '2100000000',
 'incomeTaxExpense': '213000000',
 'interestAndDebtExpense': '633000000',
 'netIncomeFromContinuingOperations': '2156000000',
 'comprehensiveIncomeNetOfTax': '1909000000',
 'ebit': '2733000000',
 'ebitda': '3283000000',
 'netIncome': '1887000000'}

#### Yahoo Finance

In [2]:
import yfinance as yf

# ticker = yf.Ticker("UBER")
# print(ticket.info)

# Get annual financial statements
# print(ticker.financials)  # Income statement
# print(ticker.balance_sheet)
# print(ticker.cashflow)

In [24]:
yf_income_statements = []
for date in ticker.financials:
    yf_income_statements.append(ticker.financials[date])
yf_income_statements.pop()  # pop since 2020 has NaN everywhere

yf_balance_sheets = []
for date in ticker.balance_sheet:
    yf_balance_sheets.append(ticker.balance_sheet[date])
yf_balance_sheets.pop()

yf_cash_flows = []
for date in ticker.cashflow:
    yf_cash_flows.append(ticker.cashflow[date])
yf_cash_flows.pop()

Free Cash Flow                             NaN
Repurchase Of Capital Stock                0.0
Repayment Of Debt                          NaN
Issuance Of Debt                           NaN
Issuance Of Capital Stock                  NaN
                                          ... 
Earnings Losses From Equity Investments    NaN
Gain Loss On Investment Securities         NaN
Net Foreign Currency Exchange Gain Loss    NaN
Gain Loss On Sale Of Business              NaN
Net Income From Continuing Operations      NaN
Name: 2020-12-31 00:00:00, Length: 65, dtype: object

### Forecast FCF
#### Revenue, Cost of Revenue, Operating Expenses

In [25]:
revenues = [int(st["Total Revenue"]) for st in yf_income_statements][::-1]
revenue_growth_rates = []


# REVENUE
print("Revenue growth rate:")
for i in range(1, len(revenues)):
    growth_rate = revenues[i] / revenues[i - 1] - 1
    revenue_growth_rates.append(growth_rate)
    print(f"\t{growth_rate * 100:.2f} %")

# Choose the last one to forecast next
forecasted_revenues = [revenues[-1] * (1 + revenue_growth_rates[-1])]
for _ in range(2):
    forecasted_revenues.append(forecasted_revenues[-1] * (1 + revenue_growth_rates[-1]))
print(f"Forcasted revenues: {forecasted_revenues}")


# COST of REVENUE (as % of Revenue)
cogs = [int(st["Cost Of Revenue"]) for st in yf_income_statements][::-1]
cogs_to_revenue = []
print("\nCost of Revenue to Revenue:")
for i in range(len(cogs)):
    ratio = cogs[i] / revenues[i]
    cogs_to_revenue.append(ratio)
    print(f"\t{ratio * 100:.2f} %")
    
# Choose the last one to forecast next 
forecasted_costs_of_revenue = []
for revenue in forecasted_revenues:
    forecasted_costs_of_revenue.append(revenue * cogs_to_revenue[-1])
print(f"Forecasted Costs of Revenue: {forecasted_costs_of_revenue}")

# OPERATING EXPENSES (as % of Revenue)
operating_expenses = [int(st["Operating Expense"]) for st in yf_income_statements][::-1]
operating_expenses_to_revenue = []
print("\nOperating Expenses to Revenue:")
for i in range(len(operating_expenses)):
    ratio = operating_expenses[i] / revenues[i]
    operating_expenses_to_revenue.append(ratio)
    print(f"\t{ratio * 100:.2f} %")
    
# Choose the last one to forecast next 
forecasted_operating_expenses = []
for revenue in forecasted_revenues:
    forecasted_operating_expenses.append(revenue * operating_expenses_to_revenue[-1])
print(f"Forecasted Operating Expenses: {forecasted_operating_expenses}")

Revenue growth rate:
	82.62 %
	16.95 %
	17.96 %
Forcasted revenues: [51878020546.65916, 61197167125.371544, 72190365490.18507]

Cost of Revenue to Revenue:
	53.57 %
	61.67 %
	60.24 %
	60.60 %
Forecasted Costs of Revenue: [31438472090.340923, 37085945269.41373, 43747906468.66438]

Operating Expenses to Revenue:
	68.39 %
	44.08 %
	36.79 %
	33.03 %
Forecasted Operating Expenses: [17137748021.780533, 20216300059.0613, 23847870067.793186]


#### Operating Profit

In [26]:
forecasted_operating_profit = []
for i in range(len(forecasted_revenues)):
    forecasted_operating_profit.append(forecasted_revenues[i] - forecasted_costs_of_revenue[i] - forecasted_operating_expenses[i])
print(f"Forecasted Operating Profit: {forecasted_operating_profit}")

Forecasted Operating Profit: [3301800434.5377007, 3894921796.8965187, 4594588953.727505]


#### Tax Rate

In [27]:
tax_rates = []
print("Historical Tax Rate:")
for st in yf_income_statements[::-1]:
    tax_rates.append(st["Tax Rate For Calcs"])
    print(f"\t{st["Tax Rate For Calcs"] * 100} %")
    
forecasted_tax_rates = [tax_rates[-1]] * 3
print(f"Forecasted Tax Rates: {forecasted_tax_rates}")

Historical Tax Rate:
	27.0 %
	1.9 %
	9.2 %
	21.0 %
Forecasted Tax Rates: [0.21, 0.21, 0.21]


#### Capital Expenditure

In [28]:
cap_ex = []
print("Historical CapEx:")
for cf in yf_cash_flows:
    cap_ex.append(-cf["Capital Expenditure"])
    print(f"\t{cf["Capital Expenditure"]}")

cap_ex_growth_rate = cap_ex[-1] / cap_ex[-2] - 1
forecasted_cap_ex = [cap_ex[-1] * (1 + cap_ex_growth_rate)]
for _ in range(2):
    forecasted_cap_ex.append(forecasted_cap_ex[-1] * (1 + cap_ex_growth_rate))
print(f"Forecasted CapEx: {forecasted_cap_ex}")

Historical CapEx:
	-242000000.0
	-223000000.0
	-252000000.0
	-298000000.0
Forecasted CapEx: [352396825.3968254, 416723230.0327538, 492791756.1498438]


#### Working Capital Change

In [30]:
working_capitals = []
print("Historical Working Capital")
for bs in yf_balance_sheets:
    working_capitals.append(bs["Working Capital"])
    print(f"\t{bs["Working Capital"]}")
recent_working_capital_change = working_capitals[-1] - working_capitals[-2]
print(recent_working_capital_change)

Historical Working Capital
	769000000.0
	1843000000.0
	396000000.0
	-205000000.0
-601000000.0


### Cost of Equity

In [12]:
# treasury_yield_resp = requests.get(f"https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=monthly&maturity=10year&apikey={AV_API_KEY}").json()
# overview_resp = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={TICKER}&apikey={AV_API_KEY}").json()

In [14]:
risk_free_rate = float(treasury_yield_resp["data"][0]["value"]) / 100  # treasury bonds rate
beta = float(overview_resp["Beta"])
expected_market_return = 0.1  # historical
market_risk_premium = expected_market_return - risk_free_rate

expected_return = risk_free_rate + beta * market_risk_premium  # cost of equity
expected_return

0.11900680000000001

### Cost of Debt

In [33]:
average_debt = (int(balance_sheets_resp["annualReports"][0]["shortLongTermDebtTotal"]) + int(balance_sheets_resp["annualReports"][1]["shortLongTermDebtTotal"])) / 2

cost_of_debt = (int(income_statements_resp["annualReports"][0]["interestAndDebtExpense"])) / average_debt
cost_of_debt

0.06690978278103694

### WACC

In [35]:
cost_of_equity = expected_return
tax_rate = int(income_statements_resp["annualReports"][0]["incomeTaxExpense"]) / int(income_statements_resp["annualReports"][0]["incomeBeforeTax"])
equity = int(balance_sheets_resp["annualReports"][0]["totalShareholderEquity"])

equity_weight = equity / (equity + average_debt)
debt_weight = average_debt / (equity + average_debt)

wacc = equity_weight * cost_of_equity + debt_weight * cost_of_debt * (1 - tax_rate)
wacc

0.0921076417820669